# 特徴量エンジニアリング（欠損値）

In [ ]:
!git clone https://github.com/nejumi/fe_workshop.git

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from pandas import DataFrame, Series
pd.set_option('display.max_columns', 100)

In [ ]:
# collaboratoryにLendingClub50000.csvをアップロードする。
from google.colab import files
#uploaded = files.upload() # ローカルからアップロードできるが今回は使用しない。

In [4]:
# ファイルを読み込む
df = pd.read_csv('fe_workshop/dataset/LendingClub50000.csv')

In [5]:
df.head()

,ID,member_id,year,month,loan_amount,purpose_type,purpose_detail,title,emp_length,home_ownership,annual_inc,zip_code,addr_state,dti,grade,sub_grade,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,bad_loan
0,1529851,1793711,2012,9,16000.0,debt_consolidation,Debt Consolidation / Final Wedding Exp,PwC,3.0,RENT,90400.0,080xx,NJ,23.72,A,A4,0.0,Oct-2003,0.0,NaN,NaN,21.0,0.0,3946.0,22.4,42.0,f,0.0,NaN,INDIVIDUAL,0.0,0.0,164787.0,False
1,1824764,2126933,2012,11,3600.0,debt_consolidation,Crush Credit Cards,Morgan Stanley Smith Barney,2.0,RENT,42500.0,217xx,MD,17.34,B,B3,0.0,Dec-2004,0.0,NaN,NaN,16.0,0.0,10927.0,63.5,30.0,f,0.0,NaN,INDIVIDUAL,0.0,8806.0,96627.0,False
2,403548,442721,2009,5,19200.0,wedding,Consolidate debt and pay for wedding,Aggregate Knowledge,1.0,RENT,95000.0,940xx,CA,7.58,B,B4,0.0,Mar-1999,0.0,NaN,NaN,11.0,0.0,14006.0,20.3,26.0,f,0.0,NaN,INDIVIDUAL,0.0,NaN,NaN,False
3,646411,799671,2011,1,21000.0,credit_card,Wells BofA Credit Card Refi,Emerson Process Management,1.0,RENT,85000.0,926xx,CA,17.07,B,B5,0.0,Jun-1999,1.0,NaN,NaN,12.0,0.0,36280.0,47.6,25.0,f,0.0,NaN,INDIVIDUAL,0.0,NaN,NaN,False
4,552526,711946,2010,7,4000.0,other,Debt Consolidation,Home Depot,1.0,RENT,63500.0,303xx,GA,19.50,C,C4,1.0,Oct-2002,2.0,23.0,NaN,16.0,0.0,595.0,11.4,24.0,f,0.0,NaN,INDIVIDUAL,0.0,NaN,NaN,False


In [28]:
# 一行ごとの欠損値の合計数を計算する。
nan_sum = df.isnull().sum(axis=1)
nan_sum.head()

0    3
1    3
2    5
3    5
4    4
dtype: int64

In [29]:
# 一行ごとの欠損値のパターンを文字列として取得する。
nan_pattern = df.apply(lambda x: ''.join(x.isnull().astype(int).astype(str)), axis=1)
nan_pattern.head()

0    0000000000000000000110000000100000
1    0000000000000000000110000000100000
2    0000000000000000000110000000100110
3    0000000000000000000110000000100110
4    0000000000000000000010000000100110
dtype: object

In [30]:
# 元データに追加する。
df['nan_sum'] = nan_sum
df['nan_pattern'] = nan_pattern

In [31]:
df.head()

,ID,member_id,year,month,loan_amount,purpose_type,purpose_detail,title,emp_length,home_ownership,annual_inc,zip_code,addr_state,dti,grade,sub_grade,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,bad_loan,nan_sum,nan_pattern
0,1529851,1793711,2012,9,16000.0,debt_consolidation,Debt Consolidation / Final Wedding Exp,PwC,3.0,RENT,90400.0,080xx,NJ,23.72,A,A4,0.0,Oct-2003,0.0,NaN,NaN,21.0,0.0,3946.0,22.4,42.0,f,0.0,NaN,INDIVIDUAL,0.0,0.0,164787.0,False,3,0000000000000000000110000000100000
1,1824764,2126933,2012,11,3600.0,debt_consolidation,Crush Credit Cards,Morgan Stanley Smith Barney,2.0,RENT,42500.0,217xx,MD,17.34,B,B3,0.0,Dec-2004,0.0,NaN,NaN,16.0,0.0,10927.0,63.5,30.0,f,0.0,NaN,INDIVIDUAL,0.0,8806.0,96627.0,False,3,0000000000000000000110000000100000
2,403548,442721,2009,5,19200.0,wedding,Consolidate debt and pay for wedding,Aggregate Knowledge,1.0,RENT,95000.0,940xx,CA,7.58,B,B4,0.0,Mar-1999,0.0,NaN,NaN,11.0,0.0,14006.0,20.3,26.0,f,0.0,NaN,INDIVIDUAL,0.0,NaN,NaN,False,5,0000000000000000000110000000100110
3,646411,799671,2011,1,21000.0,credit_card,Wells BofA Credit Card Refi,Emerson Process Management,1.0,RENT,85000.0,926xx,CA,17.07,B,B5,0.0,Jun-1999,1.0,NaN,NaN,12.0,0.0,36280.0,47.6,25.0,f,0.0,NaN,INDIVIDUAL,0.0,NaN,NaN,False,5,0000000000000000000110000000100110
4,552526,711946,2010,7,4000.0,other,Debt Consolidation,Home Depot,1.0,RENT,63500.0,303xx,GA,19.50,C,C4,1.0,Oct-2002,2.0,23.0,NaN,16.0,0.0,595.0,11.4,24.0,f,0.0,NaN,INDIVIDUAL,0.0,NaN,NaN,False,4,0000000000000000000010000000100110


In [32]:
df.to_csv('LendingClub50000_nan.csv', index=False)

In [ ]:
# ダウンロードしたら、DataRobotでもう一度予測してみよう
files.download('LendingClub50000_nan.csv')